# Commit 15: Decision Tree - Initial & Tuning

**Objective:** Implement, tune, and evaluate a Decision Tree classifier for severe traffic accident prediction. Log all hyperparameter tuning experiments and final results to the Excel sheet.

## 1. Setup and Imports

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import json
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import sys
import os

# Add src directory to path for custom modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
import modeling_utils as mu
import preprocessing_utils as pu

# Configure Pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Define constants
PROCESSED_DATA_PATH = '../data/processed/preprocessed_data.csv'
MODEL_PATH = '../models/decision_tree_model.joblib'
EXCEL_SUMMARY_PATH = '../reports/model_performance_summary.xlsx'
MODEL_NAME = 'Decision Tree'
RANDOM_STATE = 42

## 2. Load Data

In [27]:
df = pd.read_csv(PROCESSED_DATA_PATH)
print(f"Data loaded successfully. Shape: {df.shape}")
df.head()

Data loaded successfully. Shape: (22072, 44)


,Y,X,hour,day_of_week,day,month,year,is_weekend,ROAD_EDSA,MAIN_CAUSE_Human error,MAIN_CAUSE_Other (see description),MAIN_CAUSE_Road defect,MAIN_CAUSE_Unknown,MAIN_CAUSE_Vehicle defect,COLLISION_TYPE_Angle Impact,COLLISION_TYPE_Head-On,COLLISION_TYPE_Hit Object,COLLISION_TYPE_Multiple,COLLISION_TYPE_No Collision Stated,COLLISION_TYPE_Rear-End,COLLISION_TYPE_Self-Accident,COLLISION_TYPE_Side Swipe,WEATHER_Unknown,WEATHER_clear-day,WEATHER_clear-night,WEATHER_cloudy,WEATHER_fog,WEATHER_partly-cloudy-day,WEATHER_partly-cloudy-night,WEATHER_rain,LIGHT_Unknown,LIGHT_day,LIGHT_dusk,LIGHT_night,REPORTING_AGENCY_MMDA Metrobase,REPORTING_AGENCY_MMDA Road Safety Unit,REPORTING_AGENCY_Other,season_Fall,season_Spring,season_Summer,season_Winter,desc_word_count,desc_contains_collision,SEVERITY
0,14.65771,121.01979,5,0,30,6,2014,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,30,1,0
1,14.65771,121.01979,1,0,17,3,2014,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,38,1,0
2,14.65771,121.01979,2,1,26,11,2013,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,30,1,1
3,14.65771,121.01979,13,5,26,10,2013,True,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,31,1,0
4,14.65771,121.01966,23,2,26,6,2013,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,32,1,1


## 3. Define Features (X) and Target (y)

In [29]:
# Assuming 'SEVERITY' is the target variable created in notebook 01
# and all other relevant columns are features after preprocessing in notebook 02.

if 'SEVERITY' not in df.columns:
    raise ValueError("Target variable 'SEVERITY' not found in the dataframe.")

X = df.drop('SEVERITY', axis=1)
y = df['SEVERITY']

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print(f"Target distribution: {y.value_counts(normalize=True)}")

Features (X) shape: (22072, 43)
Target (y) shape: (22072,)
Target distribution: SEVERITY
0   0.93186
1   0.06814
Name: proportion, dtype: float64


## 4. Train-Test Split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (17657, 43), y_train shape: (17657,)
X_test shape: (4415, 43), y_test shape: (4415,)


## 5. Numerical Feature Scaling

In [31]:
# Identify numerical columns (assuming they are of type float or int and not binary/boolean after one-hot encoding)
numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()

# Remove any columns that are already binary (e.g., from one-hot encoding) or shouldn't be scaled
# This step might need adjustment based on the actual feature set after preprocessing
cols_to_scale = [col for col in numerical_cols if len(X_train[col].unique()) > 2] 

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

if cols_to_scale:
    X_train_scaled[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])
    X_test_scaled[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
    print(f"Scaled columns: {cols_to_scale}")
else:
    print("No columns identified for scaling.")

X_train_scaled.head()

Scaled columns: ['Y', 'X', 'hour', 'day_of_week', 'day', 'month', 'year', 'desc_word_count']


,Y,X,hour,day_of_week,day,month,year,is_weekend,ROAD_EDSA,MAIN_CAUSE_Human error,MAIN_CAUSE_Other (see description),MAIN_CAUSE_Road defect,MAIN_CAUSE_Unknown,MAIN_CAUSE_Vehicle defect,COLLISION_TYPE_Angle Impact,COLLISION_TYPE_Head-On,COLLISION_TYPE_Hit Object,COLLISION_TYPE_Multiple,COLLISION_TYPE_No Collision Stated,COLLISION_TYPE_Rear-End,COLLISION_TYPE_Self-Accident,COLLISION_TYPE_Side Swipe,WEATHER_Unknown,WEATHER_clear-day,WEATHER_clear-night,WEATHER_cloudy,WEATHER_fog,WEATHER_partly-cloudy-day,WEATHER_partly-cloudy-night,WEATHER_rain,LIGHT_Unknown,LIGHT_day,LIGHT_dusk,LIGHT_night,REPORTING_AGENCY_MMDA Metrobase,REPORTING_AGENCY_MMDA Road Safety Unit,REPORTING_AGENCY_Other,season_Fall,season_Spring,season_Summer,season_Winter,desc_word_count,desc_contains_collision
15157,0.21912,1.18529,-0.20831,-0.95999,-0.97357,-0.70638,0.59689,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,-0.61658,0
21080,1.31439,-1.80264,0.53968,0.60400,0.97455,-0.40409,-0.18423,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,-0.39879,0
16530,-0.11286,1.27369,-1.10590,-1.48133,1.43293,0.20049,0.20633,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,-0.39879,0
1759,-1.34279,-1.06690,0.39008,0.60400,1.20374,1.10737,-0.57479,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,0.69017,1
16622,0.52281,0.90476,0.68928,-1.48133,-0.97357,0.20049,0.20633,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,-0.61658,0


## 6. Class Imbalance Handling (SMOTE)

In [32]:
print(f"Original y_train distribution: {y_train.value_counts(normalize=True)}")

# Prepare data for SMOTE: keep only numeric and boolean features
X_train_smote = X_train_scaled.select_dtypes(include=['number', 'bool'])

smote = SMOTE(random_state=RANDOM_STATE)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_smote, y_train)

print(f"Resampled X_train shape: {X_train_resampled.shape}")
print(f"Resampled y_train distribution: {y_train_resampled.value_counts(normalize=True)}")

Original y_train distribution: SEVERITY
0   0.93187
1   0.06813
Name: proportion, dtype: float64
Resampled X_train shape: (32908, 43)
Resampled y_train distribution: SEVERITY
0   0.50000
1   0.50000
Name: proportion, dtype: float64


## 7. Decision Tree Model Implementation and Hyperparameter Tuning

In [37]:
dt_classifier = DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced') # Using class_weight as a baseline strategy

# Define parameter grid for GridSearchCV
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'ccp_alpha': [0.0, 0.001, 0.01, 0.1] # Cost-complexity pruning
}
# Define parameter grid for testing meant to run quickly juts seconds
param_grid_fast = {
    'criterion': ['gini'],
    'max_depth': [None, 5],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'ccp_alpha': [0.0]
}

# Initialize GridSearchCV
# Using F1-score for evaluation as it's a good metric for imbalanced datasets
grid_search = GridSearchCV(estimator=dt_classifier, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='f1_weighted', # or 'roc_auc', 'f1_macro', etc.
                           verbose=1, 
                           n_jobs=-1, 
                           return_train_score=True)

print("Starting GridSearchCV for Decision Tree...")
start_time_tuning = time.time()
grid_search.fit(X_train_resampled, y_train_resampled) # Use resampled data for tuning
tuning_time_seconds = time.time() - start_time_tuning
print(f"GridSearchCV completed in {tuning_time_seconds:.2f} seconds.")

# Best estimator
best_dt_model = grid_search.best_estimator_
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation F1 score: {grid_search.best_score_:.4f}")

Starting GridSearchCV for Decision Tree...
Fitting 5 folds for each of 360 candidates, totalling 1800 fits
GridSearchCV completed in 136.97 seconds.
Best parameters found: {'ccp_alpha': 0.0, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10}
Best cross-validation F1 score: 0.9224


### 7.1 Log All Hyperparameter Tuning Trials to Excel

In [38]:
# Log all hyperparameter tuning trials to Excel using append_performance_record from modeling_utils.py
cv_results_df = pd.DataFrame(grid_search.cv_results_)

# Select relevant columns and prepare for logging
columns_to_log = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score', 'mean_train_score', 'std_train_score']
log_df = cv_results_df[columns_to_log].copy()
log_df.rename(columns={'params': 'Hyperparameter_Set_Tried', 
                       'mean_test_score': 'CV_Score_for_Set',
                       'std_test_score': 'CV_Std_Dev_for_Set',
                       'rank_test_score': 'CV_Rank'}, inplace=True)

log_df['Model_Name'] = MODEL_NAME + ' (Tuning Trial)'
log_df['Hyperparameter_Set_Tried'] = log_df['Hyperparameter_Set_Tried'].astype(str) # Convert dict to string for Excel
log_df['Timestamp'] = pd.Timestamp.now()
log_df['Selected_Final_Hyperparameters'] = ''
log_df['Training_Time_Seconds'] = tuning_time_seconds
log_df['Train_Precision'] = ''
log_df['Train_Recall'] = ''
log_df['Train_F1'] = ''
log_df['Train_ROC_AUC'] = ''
log_df['Test_Precision'] = ''
log_df['Test_Recall'] = ''
log_df['Test_F1'] = ''
log_df['Test_ROC_AUC'] = ''
log_df['Class_Imbalance_Strategy'] = 'SMOTE'
log_df['Notes'] = 'GridSearchCV tuning trial.'

print(f"Logging {len(log_df)} hyperparameter tuning trials to Excel...")
for _, row in log_df.iterrows():
    record = {
        'Model_Name': row['Model_Name'],
        'Timestamp': row['Timestamp'],
        'Hyperparameter_Set_Tried': row['Hyperparameter_Set_Tried'],
        'CV_Score_for_Set': row['CV_Score_for_Set'],
        'Selected_Final_Hyperparameters': row['Selected_Final_Hyperparameters'],
        'Training_Time_Seconds': row['Training_Time_Seconds'],
        'Train_Precision': row['Train_Precision'],
        'Train_Recall': row['Train_Recall'],
        'Train_F1': row['Train_F1'],
        'Train_ROC_AUC': row['Train_ROC_AUC'],
        'Test_Precision': row['Test_Precision'],
        'Test_Recall': row['Test_Recall'],
        'Test_F1': row['Test_F1'],
        'Test_ROC_AUC': row['Test_ROC_AUC'],
        'Class_Imbalance_Strategy': row['Class_Imbalance_Strategy'],
        'Notes': row['Notes']
    }
    mu.append_performance_record(EXCEL_SUMMARY_PATH, record, sheet_name='DecisionTree_CV_Trials')
print("Hyperparameter tuning trials logged.")
log_df.head()

Logging 360 hyperparameter tuning trials to Excel...
Hyperparameter tuning trials logged.


,Hyperparameter_Set_Tried,CV_Score_for_Set,CV_Std_Dev_for_Set,CV_Rank,mean_train_score,std_train_score,Model_Name,Timestamp,Selected_Final_Hyperparameters,Training_Time_Seconds,Train_Precision,Train_Recall,Train_F1,Train_ROC_AUC,Test_Precision,Test_Recall,Test_F1,Test_ROC_AUC,Class_Imbalance_Strategy,Notes
0,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.91681,0.03950,18,0.99991,0.00005,Decision Tree (Tuning Trial),2025-05-16 19:49:46.610383,,136.97009,,,,,,,,,SMOTE,GridSearchCV tuning trial.
1,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.91686,0.04158,17,0.98855,0.00450,Decision Tree (Tuning Trial),2025-05-16 19:49:46.610383,,136.97009,,,,,,,,,SMOTE,GridSearchCV tuning trial.
2,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.91764,0.04502,15,0.97523,0.00764,Decision Tree (Tuning Trial),2025-05-16 19:49:46.610383,,136.97009,,,,,,,,,SMOTE,GridSearchCV tuning trial.
3,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.91723,0.04517,16,0.98250,0.00578,Decision Tree (Tuning Trial),2025-05-16 19:49:46.610383,,136.97009,,,,,,,,,SMOTE,GridSearchCV tuning trial.
4,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.91839,0.04412,14,0.98014,0.00669,Decision Tree (Tuning Trial),2025-05-16 19:49:46.610383,,136.97009,,,,,,,,,SMOTE,GridSearchCV tuning trial.


## 8. Evaluate Best Model

In [23]:
print("Evaluating the best Decision Tree model on training and test sets...")

# Ensure feature columns match those used during fitting (SMOTE)
smote_feature_cols = X_train_smote.columns.tolist()

# Predictions on the original training set (scaled, not resampled), using only SMOTE columns
y_train_pred = best_dt_model.predict(X_train_scaled[smote_feature_cols])
y_train_proba = best_dt_model.predict_proba(X_train_scaled[smote_feature_cols])[:, 1]

y_test_pred = best_dt_model.predict(X_test_scaled[smote_feature_cols])
y_test_proba = best_dt_model.predict_proba(X_test_scaled[smote_feature_cols])[:, 1]

# Calculate metrics using compute_classification_metrics from modeling_utils.py
train_metrics = mu.compute_classification_metrics(y_train, y_train_pred, y_train_proba)
test_metrics = mu.compute_classification_metrics(y_test, y_test_pred, y_test_proba)

print("\nTrain Set Metrics (Original - Scaled, Not Resampled):")
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nTest Set Metrics:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot ROC Curve for Test Set
if hasattr(mu, 'plot_roc_curve'):
    mu.plot_roc_curve(y_test, y_test_proba, model_name=MODEL_NAME, set_name='Test')
    plt.show()

# Plot Confusion Matrix for Test Set
if hasattr(mu, 'plot_confusion_matrix'):
    mu.plot_confusion_matrix(y_test, y_test_pred, model_name=MODEL_NAME, set_name='Test')
    plt.show()

Evaluating the best Decision Tree model on training and test sets...

Train Set Metrics (Original - Scaled, Not Resampled):
Precision: 0.9998
Recall: 0.9998
F1: 0.9998
ROC_AUC: nan

Test Set Metrics:
Precision: 0.8784
Recall: 0.8521
F1: 0.8647
ROC_AUC: nan


## 9. Log Final Model Performance to Excel

In [ ]:
print("Logging final model performance to Excel...")
final_record = {
    'Model_Name': MODEL_NAME + ' (Final Tuned)',
    'Timestamp': pd.Timestamp.now().isoformat(),
    'Hyperparameter_Set_Tried': '',
    'CV_Score_for_Set': '',
    'Selected_Final_Hyperparameters': json.dumps(grid_search.best_params_),
    'Training_Time_Seconds': tuning_time_seconds,
    'Train_Precision': train_metrics.get('Precision'),
    'Train_Recall': train_metrics.get('Recall'),
    'Train_F1': train_metrics.get('F1'),
    'Train_ROC_AUC': train_metrics.get('ROC_AUC'),
    'Test_Precision': test_metrics.get('Precision'),
    'Test_Recall': test_metrics.get('Recall'),
    'Test_F1': test_metrics.get('F1'),
    'Test_ROC_AUC': test_metrics.get('ROC_AUC'),
    'Class_Imbalance_Strategy': 'SMOTE',
    'Notes': 'Used SMOTE on training data for tuning. Evaluation on original train set (scaled) and test set (scaled).'
}
mu.append_performance_record(EXCEL_SUMMARY_PATH, final_record, sheet_name='Model_Summaries')
print("Final model performance logged.")

Logging final model performance to Excel...
Final model performance logged.


## 10. Save the Best Model

In [25]:
print(f"Saving the best Decision Tree model to {MODEL_PATH}")
joblib.dump(best_dt_model, MODEL_PATH)
print("Model saved successfully.")

# Also save the scaler if it was used and not saved centrally before
if cols_to_scale:
    scaler_path = '../models/decision_tree_scaler.joblib'
    joblib.dump(scaler, scaler_path)
    print(f"Scaler saved to {scaler_path}")

Saving the best Decision Tree model to ../models/decision_tree_model.joblib
Model saved successfully.
Scaler saved to ../models/decision_tree_scaler.joblib


## 11. Documentation and Summary of Results

### Model: Decision Tree

**Preprocessing Steps Applied:**
1. Loaded preprocessed data from `preprocessed_data.csv`.
2. Features (X) and target (y = `is_severe_accident`) defined.
3. Data split into training (80%) and testing (20%) sets, stratified by target.
4. Numerical features scaled using `StandardScaler` (fit on training data, applied to train and test).
5. Class imbalance in the training set handled using `SMOTE` (applied only to the scaled training data).

**Hyperparameter Tuning:**
- **Method:** `GridSearchCV` with 5-fold cross-validation.
- **Scoring Metric for Tuning:** `f1_weighted`.
- **Parameter Grid Searched:**
  - `criterion`: ['gini', 'entropy']
  - `max_depth`: [None, 5, 10, 15, 20]
  - `min_samples_split`: [2, 5, 10]
  - `min_samples_leaf`: [1, 2, 5]
  - `ccp_alpha`: [0.0, 0.001, 0.01, 0.1]
- **Best Hyperparameters Found:** (To be filled after execution - `grid_search.best_params_`)
- **Best CV F1 Score:** (To be filled after execution - `grid_search.best_score_`)
- **Tuning Time:** (To be filled after execution - `tuning_time_seconds`)
- **Logging:** All hyperparameter combinations tried and their CV scores were logged to the `Hyperparameter_Tuning_Log` sheet in `model_performance_summary.xlsx`. The final selected model's details and performance metrics were logged to the `Model_Performance_Summary` sheet.

**Evaluation Metrics (Best Tuned Model):**
*(To be filled after execution with actual values from `train_metrics` and `test_metrics`)*
| Metric         | Train (Original, Scaled) | Test (Scaled) |
|----------------|--------------------------|---------------|
| Accuracy       |                          |               |
| Precision      |                          |               |
| Recall         |                          |               |
| F1 Score       |                          |               |
| ROC AUC        |                          |               |

**Model Persistence:**
- The best tuned Decision Tree model was saved to: `../models/decision_tree_model.joblib`
- The scaler used was saved to: `../models/decision_tree_scaler.joblib` (if applicable)

**Observations & Next Steps:**
*(To be filled after execution. Comment on performance, potential overfitting/underfitting, comparison to other models if available, and any specific insights from the Decision Tree model.)*